# P-Biome 16S rRNA Pipeline
### Pipeline will be documented step by step , but everything is on a Snakefile so it is fully automated. Or you can run it step by step in the cells that start with '!qiime ....'

# Dependencies needed
### 1.  [Python](https://www.python.org/) ≥3.3 
### 2. [Miniconda enviroment](https://conda.io/miniconda.html) - will make everything easier later
### 3. [QIIME2](https://docs.qiime2.org/2017.9/install/native/#install-miniconda) via conda instalation 
### 4. [Snakemake](http://snakemake.readthedocs.io/en/stable/tutorial/setup.html)

# The Following checks that dependencies are in your path and what their versions are

In [ ]:
# %%bash 
qiime --version #q2cli version 2017.8.0
python --version #Python 3.5.4 :: Continuum Analytics, Inc.
conda --version #conda 4.3.27
snakemake --version #3.13.3
# if you are checking this in your environment do not add the '%%bash' command. This is only needed for this tutorial 

# Getting started with the proper environment

In [14]:
%%bash
ls taxonomic_classifier/
# check to see that the classifier is in there. Name should pop up like below

classifier.qza


In [15]:
%%bash
ls R1_files/
#all your foward files should be uploaded to R1_files folder, ignore the extra right now, but notice that all files 
# have the sample same (see below) and that they all end in .gz . If your  files end in .fastq use the following to convert
# gzip R1_files/*.fastq
# then everything should end in gz

D1_S4_L001_R1_001.fastq.gz
D2_S12_L001_R1_001.fastq.gz
D8_S60_L001_R1_001.fastq.gz
extra
Mock_S280_L001_R1_001.fastq.gz


### Make sure your mapping file is properly filled. Example of mapping file can be found in [qiime2](https://docs.qiime2.org/2.0.6/tutorials/moving-pictures/) tutorial or you can get help [here](http://keemei.qiime.org/)
### Ask if you are not sure, this becomes very important throughout our analysis so we want to make sure this looks right. All files must be represented in mapping file. Then upload your mapping file to the Gut_microbiome_I directory. After this most everything should be ready to go.

In [1]:
%%bash
ls
# this is what your folder should look like now, names must look exacly like this

Final.ipynb
Mapping_file.txt
R1_files
Snakefile
dag.svg
taxonomic_classifier


# Let's start

In [ ]:
# make sure you are in your macqiime environent
source activate qiime2-2017.8
# also if you are using a cluster or cloud computer use screen incase you get disconnected everything still keeps working
screen
# the following is a step-by-step of what is going on with the snakefile

# Make_artifacts - Imports fastq reads that are ALREADY demultiplexed into a qiime artifact
## start by using this code, this will allow you to cizualize your sequences before denoising later
### ```snakemake --until demux.qzv```

In [ ]:
rule make_artifacts:
    input:
        "R1_files"
    output:
        "demux-single-end.qza"
    message:
        "Welcome to the Persephone Biome 16S rRNA Pipeline. Make sure all your dependencies are meet and that you have all the proper files in their proper folder. If unsure look at the jupyter notebook tutorial."
    shell:
        "qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path {input} \
  --source-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path {output}"

In [ ]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path R1_files \
  --source-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-single-end.qza

## Summarize demultiplexed files - a summary of sequencing depth in each of your files

In [ ]:
rule summarize_demux:
    input:
        "demux-single-end.qza"
    output:
        "demux.qzv"
    shell:
        "qiime demux summarize \
  --i-data {input} \
  --o-visualization {output}" 

In [ ]:
!qiime demux summarize \
  --i-data demux-single-end.qza \
  --o-visualization demux.qzv

# This is around the point where you should stop and check the quality of your files. You are going to need to know this for the following step. We need to know how good or bad our sequences are and where it is that we need to trim. 
`

In [18]:
# view your output
!qiime tools view demux.qzv
# see below, doesn't look great so we will try our best in the filtering department

Usage: qiime tools view [OPTIONS] VISUALIZATION_PATH

Error: Visualization viewing is currently not supported in headless environments. You can view Visualizations (and Artifacts) at https://view.qiime2.org, or move the Visualization to an environment with a display and view it with `qiime tools view`.


![title](demuxpic.png)


In [ ]:
# YOU PROBABLY ONLY WANT TO GET UP TO THIS POINT AT FIRST. THIS WAY YOU CAN EVALUATE YOUR DATA AND CHOOSE THE BEST WAY
# TO TRIM YOUR DATA IN BOTH THE 5' AND 3' END IF NEEDED. YOU ARE TRYING TO LOSE ENDS THAT HAVE THE LOWEST QUALITY OF SEQUENCES
#COMPARED TO THE REST OF YOUR DATA. USE THIS CODE TO RUN UP TO HERE!!


# DADA2 denoise - Assumed you have looked at the demux.qzv file and determined what the best parameters are now you will run this code.The following denoising step is subject to change. Make sure you look at the above graph and change the parameters as needed. 

### if you need to modify the code below (as this is currently the default) open the Snakefile and modify it.

In [ ]:
#Run this code now to start your analysis up until you make the phyogenetic tree. Now keep in mind you only need to do this 
#if you haven;t aready determined the optimal parameters for your data.
snakemake --until rooted-tree.qza
# if you will use the default value for the core diveristy metrics (look below) than use this instead to just finish



In [ ]:
rule denoise_single:
    input:
        "demux-single-end.qza"
    output:
        "rep-seqs-dada2.qza"
    message:
        "This is a very long step. Your sequences are being analyzed for sequencing errors, duplicates and chimeras. Will remove spurious OTUs and reduce inflation commonly associated with OTU clustering. Will also reduce dataset size so don't be alarmed if you notice this. It is necessary. Make sure you are in a 'screen' like environment or do not close your computer. Depending on the data size and computer strength this could take little to a long time. Don't be worried if it takes a while."
    shell:
        "qiime dada2 denoise-single \
    --i-demultiplexed-seqs {input} \
    --p-trim-left 20 --p-trunc-len 220 --p-n-threads 0 \
    --o-representative-sequences {output}\
    --o-table table-dada2.qza"

In [ ]:
!qiime dada2 denoise-single \
    --i-demultiplexed-seqs demux-single-end.qza \
    --p-trim-left 12 --p-trunc-len 220 \
    --o-representative-sequences rep-seqs-dada2.qza \
    --o-table table-dada2.qza

# Summarize -  feature table and rep seqs

In [ ]:
rule table_summary:
    input:
        "table-dada2.qza"
    output:
        "table-dada2.qzv"

    shell:
        "qiime feature-table summarize \
  --i-table  {input} \
  --o-visualization  {output} \
  --m-sample-metadata-file Mapping_file.txt"

In [ ]:
!qiime feature-table summarize \
  --i-table  table-dada2.qza \
  --o-visualization  table-dada2.qzv \
  --m-sample-metadata-file Mapping_file.txt

In [ ]:
rule rep_summary:
    input:
        "rep-seqs-dada2.qza"
    output:
        "rep-seqs-dada2.qzv"
    shell:
        "qiime feature-table tabulate-seqs \
  --i-data  {input} \
  --o-visualization  {output}"

In [ ]:
!qiime feature-table tabulate-seqs \
  --i-data  rep-seqs-dada2.qza \
  --o-visualization  rep-seqs-dada2.qzv

# Generate a tree for phylogenetic analysis - multistep process to build a phylogenetic tree for downstream analysis

In [ ]:
rule phylogenetic_tree:
    input:
        "rep-seqs-dada2.qza"
    output:
        "aligned-rep-seqs.qza"
    shell:
        "qiime alignment mafft \
  --i-sequences {input} \
  --o-alignment {output}"

In [ ]:
rule alignment:
    input:
        "aligned-rep-seqs.qza"
    output:
        "masked-aligned-rep-seqs.qza"
    shell:
    "qiime alignment mask \
  --i-alignment {input} \
  --o-masked-alignment {output}"

In [ ]:
rule phylogeny_fasttree:
    input:
        "masked-aligned-rep-seqs.qza"
    output:
        "unrooted-tree.qza"
    shell:
        "qiime phylogeny fasttree \
  --i-alignment {input} \
  --o-tree {output}"

In [ ]:
rule midpoint:
    input:
        "unrooted-tree.qza"
    output:
        "rooted-tree.qza"
    shell:
        "qiime phylogeny midpoint-root \
  --i-tree {input} \
  --o-rooted-tree {output}"

In [ ]:
!qiime alignment mafft \
  --i-sequences rep-seqs-dada2.qza\
  --o-alignment aligned-rep-seqs.qza

!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

!qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza

!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

# Core diversity metrics - will do alpha and beta diversity metrics of all sorts.
### Before you start you should take a good look at your table.qzv if you decide you want to change the following parameters. Data must be rarefied prior to doing alpha and beta so no matter what you have to temporarily put some data aside. Look at the sequence count in your table.qzv file and look at the sample with the lowest depth and you should rarefy around there. Default value right now is a depth of 2500 sequences.

In [ ]:
rule alpha_beta_metrics:
    input:
        tree="rooted-tree.qza",
        table="table-dada2-mock.qza"
    output:
        "core-metrics-results-mock"
    shell: 
        "qiime diversity core-metrics \
  --i-phylogeny {input.tree} \
  --i-table {input.table} \
  --p-sampling-depth 2500 \
  --output-dir example_paired_end_demultiplex/se-hirocon/core-metrics-results"

In [ ]:
!qiime diversity core-metrics \
  --i-phylogeny rooted-tree.qza \
  --i-table table-dada2-mock.qza \
  --p-sampling-depth 2500 \
  --output-dir example_paired_end_demultiplex/se-hirocon/core-metrics-results

# Alpha beta visualization

In [ ]:
rule alpha_visualization:
    #input:
    #    "core-metrics-phylogenetic/{sample}_vector.qza"
    output:
        "visualization/alpha"
    shell:
        """
        echo "Running Alpha Visualization"
        mkdir {output}

        qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/evenness_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/evenness_vector.qzv
        qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/faith_pd_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/faith_pd_vector.qzv
        qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/observed_otus_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/observed_otus_vector.qzv
        qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/shannon_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/shannon_vector.qzv

        """

In [ ]:
%%bash
mkdir visualization/alpha

In [ ]:
!qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/evenness_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/evenness_vector.qzv
!qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/faith_pd_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/faith_pd_vector.qzv
!qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/observed_otus_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/observed_otus_vector.qzv
!qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/shannon_vector.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/alpha/shannon_vector.qzv


In [ ]:
rule beta_visualization:
    output:
        "visualization/beta"
    shell:
        """
        echo "Running Beta Group Significance"
        mkdir {output}
        qiime emperor plot --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/unweighted-unifrac-emperor.qzv
        qiime emperor plot --i-pcoa core-metrics-results/weighted_unifrac_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/weighted-unifrac-emperor.qzv
        qiime emperor plot --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/bray_curtis-emperor.qzv
        qiime emperor plot --i-pcoa core-metrics-results/jaccard_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/jaccard-emperor.qzv
        """

In [ ]:
%%bash
mkdir visualization/beta

In [ ]:
!qiime emperor plot --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/unweighted-unifrac-emperor.qzv
!qiime emperor plot --i-pcoa core-metrics-results/weighted_unifrac_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/weighted-unifrac-emperor.qzv
!qiime emperor plot --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/bray_curtis-emperor.qzv
!qiime emperor plot --i-pcoa core-metrics-results/jaccard_pcoa_results.qza --m-metadata-file sample-metadata.tsv --o-visualization visualization/beta/jaccard-emperor.qzv

# Taxonomic Analysis - taxonomic classification using classifier.qza and then make a taxa bar plot. Data is not rarefied here.

In [ ]:
rule feature_classifier:
    input:
        classifier= "taxonomic_classifier/classifier.qza",
        repseq="rep-seqs-dada2.qza"
    output:
        "taxonomy.qza"
    shell:
        """
        echo "Running feature classifier"
        qiime feature-classifier classify-sklearn --i-classifier {input.classifier} --i-reads {input.repseq} --o-classification {output}
        """

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-classifier taxonomic_classifier/classifier.qza \
    --i-reads rep-seqs-dada2.qza \
    --o-classification taxonomy.qza

In [ ]:
rule taxa_barplot:
    input:
        "taxonomy.qza"
    output:
        "visualization/taxa"
    shell:
        """
        mkdir {output}
        qiime taxa barplot --i-table table-dada2.qza \
        --i-taxonomy taxonomy.qza --m-metadata-file Mapping_file.txt  \
        --o-visualization visualization/taxa/taxa-barplot.qzv

        """

In [ ]:
%%bash
mkdir visualization/taxa

In [ ]:
!qiime taxa barplot --i-table table-dada2.qza \
    --i-taxonomy taxonomy.qza --m-metadata-file Mapping_file.txt  \
    --o-visualization visualization/taxa/taxa-barplot.qzv